## Subplaces back to NYC boroughs

In [1]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [2]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
import os
import plotly.express as px
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [3]:
subplaces = r'data/shapefiles/nyc_subplace_centroid.geojson'
subplaces = gpd.read_file(subplaces)
subplaces.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,POINT (-73.12283 41.52707)
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,POINT (-73.06185 41.22575)
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,POINT (-73.13503 41.43129)
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,POINT (-72.79470 41.25792)
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,POINT (-72.62809 41.34013)


In [4]:
df = pd.read_csv(r'data/SG_origsubxnyc_01-10-2020.csv')
decimals = 0

df = df.rename(columns = {'orig_subpl_str':'id'})
df['id'] = df['id'].str[1:]
list_date = df.columns.to_list()
list_date = list_date[2:]
# melt so we have boroughs as a column.
df = pd.melt(df, id_vars=['id','date_y-m-d'], value_vars = list_date)
df = df.rename(columns = {'variable':'Borough','date_y-m-d':'date'})
df['date'] = pd.to_datetime(df['date'])
# merge
df = pd.merge(subplaces, df, left_on='GEOID', right_on='id', how = "inner")
df['date'] = df["date"].dt.strftime('%m/%d/%Y')
df['value'] = df['value'].astype(float)
df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
df['value'] = df['value'].astype(int) # remove decimal
#remove rows less that are 0.
df = df.loc[df['value']>=1]
df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Returns/Device'})
# df.to_csv(f'data/returns_from_subplace.csv', index = False)
    
# returns(df)


In [ ]:
df.head()

In [5]:
df = pd.read_csv(r'data/returns_from_subplace.csv')
df = df.sort_values(by = 'Date')
df = df[df['Borough'] != 'NYC']

C:\Users\steve\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,Name,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry,id,Date,Borough,Returns/Device
0,1,9,9.0,46940.0,213459.0,900946940,Middlebury,Middlebury town,43,T1,G4040,720.0,78700.0,NaN,A,45986818.0,1786816.0,41.524691,-73.123016,NaN,NaN,NaN,NaN,0.321040,0.005154,47773634.0,18.44540,9.009469e+08,-234.0,POINT (-73.12282609979773 41.52707296499348),900946940,01/01/2020,BX,92
345581,299,34,13.0,22385.0,1729722.0,3401322385,Fairfield,Fairfield township,44,T1,G4040,NaN,NaN,NaN,A,26271159.0,492809.0,40.884584,-74.303232,NaN,NaN,NaN,NaN,0.374943,0.002859,26763968.0,10.33357,3.401322e+09,-1027.0,POINT (-74.30408796446306 40.88283020678721),3401322385,01/01/2020,BK,87
43671,59,9,5.0,80490.0,213527.0,900580490,Watertown,Watertown town,43,T1,G4040,720.0,78700.0,NaN,A,75121154.0,1312297.0,41.617246,-73.114362,NaN,NaN,NaN,NaN,0.400634,0.008257,76433451.0,29.51096,9.005805e+08,-910.0,POINT (-73.11773216745871 41.61601967842524),900580490,01/01/2020,BX,23
597700,525,36,111.0,63737.0,979435.0,3611163737,Rosendale,Rosendale town,43,T1,G4040,NaN,NaN,NaN,A,51744384.0,1985488.0,41.847090,-74.079544,NaN,NaN,NaN,NaN,0.360501,0.005825,53729872.0,20.74510,3.611164e+09,-54.0,POINT (-74.07861342854252 41.84734047157456),3611163737,01/01/2020,QN,116
345757,299,34,13.0,22385.0,1729722.0,3401322385,Fairfield,Fairfield township,44,T1,G4040,NaN,NaN,NaN,A,26271159.0,492809.0,40.884584,-74.303232,NaN,NaN,NaN,NaN,0.374943,0.002859,26763968.0,10.33357,3.401322e+09,-1027.0,POINT (-74.30408796446306 40.88283020678721),3401322385,01/01/2020,MN,419


In [9]:
fig = px.scatter_mapbox(df,# geojson = subplaces,
                     lat = 'INTPTLAT', lon = 'INTPTLON',
                     size = 'Returns/Device',
                     color ='Borough',
                     color_discrete_sequence = ['#78D3E3','#1D819A','#FFE066','#63BB69','#DE7A3F'],
                     animation_frame = 'Date',
                     animation_group = 'Borough',
                     hover_data = {'Name':True,'Date':True,'Returns/Device':True, 'INTPTLAT':False, 'INTPTLON':False, 'Borough':False},
                #     labels={'Borough':'Borough of Residence'},
                     #font = dict(family = 'Arial', color = 'white'),
                     size_max = 40)
           #          labels = 'Borough')
fig.update_layout(mapbox_zoom = 6.8,
                 mapbox_center = {'lat':40.82, 'lon':-74.25},
                 mapbox_style="carto-darkmatter",
                 width = 950,
                 height = 780, 
                 font = dict(family = 'Arial', color = 'black'),
                 title_font_color = 'white',
                 legend=dict(
#                       orientation="h",
                      title = 'Borough of Residence',
                      title_font_color = 'white',
                      bgcolor = 'black',
                      bordercolor = 'gray', borderwidth = 1,
                      font = dict(family = 'Arial', color = 'white'),
                      yanchor="top",
                      y=0.39,
                      xanchor="left",
                      x=0.02))#,
   #              margin = {'r':1,'t':1,'l':1,'b':1},
  #               legend = dict(yanchor = "top", y = 0.8, xanchor = "left", x = 0.10))
fig.add_annotation(text = "Non-NYC Residents' Trips to NYC<br>by Home Municipality",
                  align = 'left', x = 0.01, y = 0.95, showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  font = dict(family = "Arial", color = 'white', size = 19))
fig.add_annotation(text = 'January 1, 2020 to October 31, 2020',
                  align = 'left', x = 0.01, y = 0.86, showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  font = dict(family = "Arial", color = 'white', size = 13))
fig.add_annotation(text = 'Bubble size = Trips per 10,000 mobile devices',
                  align = 'right', x = 0.02, y = 0.082, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
fig.add_annotation(text = 'Click on/off boroughs in legend to compare or isolate migratory patterns',
                  align = 'right', x = 0.02, y = 0.01, showarrow=False,
                  bordercolor = None, bgcolor = None,
                  font = dict(family = 'Droid Sans', color = 'white', size = 12))
# fig.show()
fig.write_html('subplace_returns.html',auto_open = True,auto_play = False, include_plotlyjs = 'cdn')